In [2]:
pwd()
using Pkg; Pkg.activate("../../FermiCG/")
using FermiCG, NPZ
readdir()

  Activating project at `~/code/FermiCG`
┌ Info: Precompiling FermiCG [855c3b3a-d9bb-4919-8b00-0b3cfbdffb6f]
└ @ Base loading.jl:1423


34-element Vector{String}:
 ".ipynb_checkpoints"
 "C_act.molden"
 "C_act_loc.molden"
 "C_act_loc_sorted.molden"
 "C_act_loc_sorted_2.molden"
 "C_act_loc_sorted_3.molden"
 "C_doc.molden"
 "README.md"
 "Untitled.ipynb"
 "b3lyp-d3_def2svp.xyz"
 "b3lyp-d3_def2svp_S4.xyz"
 "clustered1_integrals_h0.npy"
 "clustered1_integrals_h1.npy"
 ⋮
 "cr_qubit_scf.ipynb"
 "cr_qubit_scf_integrals_h0.npz.npy"
 "cr_qubit_scf_integrals_h1.npz.npy"
 "cr_qubit_scf_integrals_h2.npz.npy"
 "cr_qubit_scf_mo_coeffs.npz.npy"
 "excitations1.xlsx"
 "scf.ipynb"
 "uhf_density.npy"
 "uhf_mo_coeffs_a.npy"
 "uhf_mo_coeffs_b.npy"
 "venv"
 "wb97x-v_def2svp.xyz"

In [3]:
using FermiCG, NPZ, JLD2


In [5]:

#load integrals from disk
h0 = npzread("clustered1_integrals_h0.npy")
h1 = npzread("clustered1_integrals_h1.npy")
h2 = npzread("clustered1_integrals_h2.npy")
ints = InCoreInts(h0, h1, h2)
ints_original = deepcopy(ints);

println(" Integrals have the following sizes: h0= ", size(h0), " h1= ", size(h1), " h2= ", size(h2))


# Define clusters - this should probably be done in the python notebook, and then just read in here
clusters_in = [
	       (1:6), # Benzene 1
	       (7:12), # Benzene 1
	       (13:18), # Benzene 1
	       (19:24), # Benzene 1
	       (25:34),   # metal
	       ]

clusters = [Cluster(i,collect(clusters_in[i])) for i = 1:length(clusters_in)]

init_fspace = [
	       (3,3),
	       (3,3),
	       (3,3),
	       (3,3),
	       (5,5)];
display(clusters)
display(init_fspace)


rdm1 = zeros(size(ints.h1))

e_cmf, U, Da, Db = FermiCG.cmf_oo(ints, clusters, init_fspace, rdm1, rdm1, verbose=0, gconv=1e-6, method="bfgs", sequential=true);

ints = FermiCG.orbital_rotation(ints_original,U);

@save "cmf_data_1.jld2" ints clusters init_fspace Da Db

5-element Vector{Cluster}:
 Cluster(1, [1, 2, 3, 4, 5, 6])
 Cluster(2, [7, 8, 9, 10, 11, 12])
 Cluster(3, [13, 14, 15, 16, 17, 18])
 Cluster(4, [19, 20, 21, 22, 23, 24])
 Cluster(5, [25, 26, 27, 28, 29, 30, 31, 32, 33, 34])

5-element Vector{Tuple{Int64, Int64}}:
 (3, 3)
 (3, 3)
 (3, 3)
 (3, 3)
 (5, 5)

 Integrals have the following sizes: h0= () h1= (34, 34) h2= (34, 34, 34, 34)
 ooCMF Iter:    1 Total= -2119.139473784612 Active= -105.879636511977 G=     3.37e-01
 ooCMF Iter:    2 Total= -2119.176614257495 Active= -105.916776984860 G=     8.96e-02
 ooCMF Iter:    3 Total= -2119.179123717736 Active= -105.919286445101 G=     4.65e-02
 ooCMF Iter:    4 Total= -2119.179929055183 Active= -105.920091782548 G=     3.38e-02
 ooCMF Iter:    5 Total= -2119.180400548608 Active= -105.920563275973 G=     3.07e-02
 ooCMF Iter:    6 Total= -2119.180830672000 Active= -105.920993399364 G=     1.88e-02
 ooCMF Iter:    7 Total= -2119.180914842149 Active= -105.921077569514 G=     7.16e-03
 ooCMF Iter:    8 Total= -2119.180928851449 Active= -105.921091578814 G=     3.16e-03
 ooCMF Iter:    9 Total= -2119.180933276258 Active= -105.921096003623 G=     2.80e-03
 ooCMF Iter:   10 Total= -2119.180937976326 Active= -105.921100703691 G=     2.15e-03
 ooCMF Iter:   11 Total= -2119.180939805756 Active= -105.92110

 * Status: success

 * Candidate solution
    Final objective value:     -2.119181e+03

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 9.85e-07 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.42e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.50e-11 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.18e-14 ≰ 0.0e+00
    |g(x)|                 = 6.89e-07 ≤ 1.0e-06

 * Work counters
    Seconds run:   269  (vs limit Inf)
    Iterations:    24
    f(x) calls:    54
    ∇f(x) calls:   54


 ooCMF Iter:   25 Total= -2119.180940409833 Active= -105.921103137198 G=     4.68e-06
*ooCMF -2119.18094041 


┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /Users/nmayhall/.julia/packages/PyCall/L0fLP/src/numpy.jl:67
